<h1 style="color:red;font-weight: 900;">Hosseini Project Source Code</h1>

<h1 style="">Import libraries</h1>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from scipy.fft import fft
import pywt
import os

<h1>Data Collection</h1>

In [ ]:
def fetch_btc_data(start_date='2018-01-01', end_date='2024-12-31'):
    btc = yf.download('BTC-USD', start=start_date, end=end_date, interval='1d')
    return btc

<h1>Data Preprocessing</h1>

In [ ]:
def preprocess_data(data):
    # Handle missing values
    data = data.fillna(method='ffill')
    
    # Select closing price
    prices = data['Close'].values.reshape(-1, 1)
    
    # Normalize data
    scaler = MinMaxScaler()
    prices_scaled = scaler.fit_transform(prices)
    
    # Split data
    train_size = int(len(prices_scaled) * 0.7)
    val_size = int(len(prices_scaled) * 0.15)
    train_data = prices_scaled[:train_size]
    val_data = prices_scaled[train_size:train_size + val_size]
    test_data = prices_scaled[train_size + val_size:]
    
    return train_data, val_data, test_data, scaler, prices

<h1>Create sequences for LSTM</h1>

In [ ]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

<h1>Frequency Analysis (FFT)</h1>

In [ ]:
def perform_fft(data):
    fft_result = fft(data)
    frequencies = np.fft.fftfreq(len(fft_result))
    return fft_result, frequencies

<h1>Aux Functions</h1>

In [ ]:
# 5. Wavelet Transform
def perform_wavelet_transform(data, wavelet='db4', level=4):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    return coeffs

# 6. Build and Train LSTM Model
def build_lstm_model(seq_length):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 7. Evaluate Model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

<h1 style="color:yellow;">Main Function</h1>

In [ ]:
def main():
    # Fetch data
    btc_data = fetch_btc_data()
    
    # Preprocess data
    train_data, val_data, test_data, scaler, raw_prices = preprocess_data(btc_data)
    
    # Create sequences
    seq_length = 60
    X_train, y_train = create_sequences(train_data, seq_length)
    X_val, y_val = create_sequences(val_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)
    
    # Reshape for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    # Frequency Analysis (FFT)
    fft_result, frequencies = perform_fft(raw_prices.flatten())
    plt.figure(figsize=(10, 6))
    plt.plot(frequencies[:len(frequencies)//2], np.abs(fft_result)[:len(frequencies)//2])
    plt.title('FFT Spectrum of BTC Prices')
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.savefig('fft_spectrum.png')
    plt.close()
    
    # Wavelet Transform
    coeffs = perform_wavelet_transform(raw_prices.flatten())
    plt.figure(figsize=(10, 6))
    for i, coeff in enumerate(coeffs):
        plt.subplot(len(coeffs), 1, i+1)
        plt.plot(coeff)
        plt.title(f'Wavelet Coefficient {i}')
    plt.tight_layout()
    plt.savefig('wavelet_transform.png')
    plt.close()
    
    # Train LSTM
    model = build_lstm_model(seq_length)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=50, batch_size=32, verbose=1)
    
    # Predict
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)
    
    # Inverse transform predictions
    train_pred = scaler.inverse_transform(train_pred)
    val_pred = scaler.inverse_transform(val_pred)
    test_pred = scaler.inverse_transform(test_pred)
    y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
    y_val_inv = scaler.inverse_transform(y_val.reshape(-1, 1))
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Evaluate
    train_mae, train_rmse, train_r2 = evaluate_model(y_train_inv, train_pred)
    val_mae, val_rmse, val_r2 = evaluate_model(y_val_inv, val_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test_inv, test_pred)
    
    print(f"Train MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R2: {train_r2:.4f}")
    print(f"Val MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}, R2: {val_r2:.4f}")
    print(f"Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R2: {test_r2:.4f}")
    
    # Plot predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test_inv, label='Actual Prices')
    plt.plot(test_pred, label='Predicted Prices')
    plt.title('LSTM Predictions vs Actual BTC Prices')
    plt.xlabel('Time')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.savefig('lstm_predictions.png')
    plt.close()
    
    # Plot raw prices
    plt.figure(figsize=(12, 6))
    plt.plot(btc_data.index, raw_prices, label='BTC Price')
    plt.title('BTC Daily Prices (2018-2024)')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.savefig('btc_price_plot.png')
    plt.close()